# Load data

In [ ]:
!pip install monty megnet

import numpy as np
from monty.json import MontyDecoder
from monty.serialization import loadfn

data = loadfn('bulk_moduli.json')
structures = data['structures']
targets = np.log10(data['bulk_moduli'])

In [ ]:
len(structures)

100

# Set up model and train

In [ ]:
from megnet.models import MEGNetModel
from megnet.data.crystal import CrystalGraph
import numpy as np

nfeat_bond = 10
r_cutoff = 5
gaussian_centers = np.linspace(0, r_cutoff + 1, nfeat_bond)
gaussian_width = 0.5
graph_converter = CrystalGraph(cutoff=r_cutoff)
model = MEGNetModel(graph_converter=graph_converter, centers=gaussian_centers, width=gaussian_width)

TypeError: Trainer.compile() got an unexpected keyword argument 'sample_weight_mode'

In [ ]:
from megnet.data.crystal import CrystalGraph
from megnet.data.graph import GaussianDistance
from megnet.models import MEGNetModel

model = MEGNetModel(10, 2, nblocks=1,
                    lr=1e-2, n1=4, n2=4, n3=4, npass=1, ntarget=1,
                    graph_converter=CrystalGraph(bond_converter=GaussianDistance(np.linspace(0, 5, 10), 0.5)))

# model = MEGNetModel(
#     graph_converter=CrystalGraph(
#         bond_converter=GaussianDistance(centers=np.linspace(0, 5, 10), width=0.5)
#     ),
#     nblocks=3,
#     nvocal=95,
#     embedding_dim=16,
#     npass=2,
#     learning_rate=1e-3,
#     ntarget=1,  # For three output values
#     lr=1e-3,
#     n1=64,
#     n2=32,
#     n3=16,
#     loss="mae",
#     metrics=["mae"]
# )

TypeError: Trainer.compile() got an unexpected keyword argument 'sample_weight_mode'

In [ ]:
model.train(structures, targets, epochs=2)

NameError: name 'model' is not defined

# Prediction

## Predict from structure

In [ ]:
from pymatgen import MPRester
mpr = MPRester()

# let's get some structure
structure = mpr.get_structure_by_material_id('mp-1143')

In [ ]:
predicted_K = 10 ** model.predict_structure(structure).ravel()
print('The predicted K for {} is {} GPa'.format(structure.formula, predicted_K[0]))

The predicted K for Al4 O6 is 1.2169744968414307 GPa
